<a href="https://colab.research.google.com/github/baeksumin/k-project/blob/main/datacollection_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
'''
1. 저장된 fullcode 목록을 불러와서 종목별 데이터 수집
2. pickle 저장

(프로젝트를 시작할 때 1번만 실행)
'''

In [1]:
import json
import pytz
import requests
import pandas as pd
from io import BytesIO
from datetime import datetime 
from datetime import timedelta
import pickle
from google.colab import drive

drive.mount('/content/drive')
output_path = '/content/drive/MyDrive/k-project/output/'
data_path = '/content/drive/MyDrive/[Shared]K-Project/'
# pickle 경로를 설정하기 위해

Mounted at /content/drive


In [2]:
# 한국날짜 설정
def kor_date():
    KST = pytz.timezone('Asia/Seoul')
    today = str(datetime.now(KST).strftime("%Y%m%d"))
    return today

In [5]:
## Load pickle - fullcode 목록
with open(data_path + 'full_code.pkl',"rb") as f: 
    fullcode = pickle.load(f)
    
# 상장폐지 상태 제외시키기
fullcode_filter = fullcode[fullcode['status'] != '상장폐지']

# fullcode열을 list로 변경
fullcode_list = list(fullcode_filter['full_code'])
print(fullcode_list[:10])

['KR7060310000', 'KR7095570008', 'KR7006840003', 'KR7054620000', 'KR7265520007', 'KR7211270004', 'KR7027410000', 'KR7282330000', 'KR7138930003', 'KR7001460005']


In [ ]:
'''
종가 시가 저가 고가 시가총액 EPS PER BPS PBR 주당배당금 배당수익률
(* 필요한 컬럼)

df2
 0   TRD_DD : 거래일자 *
 1   ISU_CD : 종목코드 *      
 2   ISU_NM : 종목명 *      
 3   MKT_NM : 시장코드 (kospi)      
 4   SECUGRP_NM : 증권구분 (주권)    
 5   TDD_CLSPRC : 종가 *  
 6   FLUC_TP_CD    
 7   CMPPRVDD_PRC : 대비 
 8   FLUC_RT : 등락률      
 9   TDD_OPNPRC : 시가 *  
 10  TDD_HGPRC  : 고가 *  
 11  TDD_LWPRC  : 저가 *  
 12  ACC_TRDVOL : 거래량   
 13  ACC_TRDVAL : 거래대금   
 14  MKTCAP : 시가총액 *

df3
 0   TRD_DD : 거래일자        
 1   TDD_CLSPRC : 종가    
 2   FLUC_TP_CD     
 3   CMPPREVDD_PRC : 대비 
 4   FLUC_RT : 등락률      
 5   EPS *           
 6   PER *           
 7   BPS *           
 8   PBR *           
 9   DPS : 주당배당금 *          
 10  DVD_YLD : 배당수익률 *
 '''

In [26]:
# 데이터 가져와서 필요 컬럼 추출 후 데이터프레임 생성하여 json으로 저장하는 함수
def make_initialdata():
    data2 = { 'bld': 'dbms/MDC/STAT/issue/MDCSTAT23902',
    'isuCd': i, # i는 KR7060310000
    'isuCd2': '',
    'strtDd': strtDd,
    'endDd': endDd,
    'share': '1',
    'money': '1',
    'csvxls_isNo': 'false',}

    r2 = requests.post(url2, data2, headers)
    j2 = json.loads(r2.text)
    df2=pd.DataFrame(j2['output'])

    data3 = { 'bld': 'dbms/MDC/STAT/standard/MDCSTAT03502',
    'isuCd': i,
    'isuCd2': i,
    'strtDd': strtDd,
    'endDd': endDd,
    'searchType': '2',
    'mktId': 'ALL',
    'csvxls_isNo': 'false',}

    r2 = requests.post(url2, data3, headers)
    j2 = json.loads(r2.text)
    df3=pd.DataFrame(j2['output'])

    df2_ = df2[['TRD_DD', 'ISU_CD', 'ISU_NM', 'TDD_CLSPRC', 'TDD_OPNPRC', 'TDD_HGPRC', 'TDD_LWPRC', 'MKTCAP']]
    df3_ = df3[['TRD_DD', 'EPS', 'PER', 'BPS', 'PBR', 'DPS', 'DVD_YLD']]
    result = pd.merge(df2_, df3_)

    result_js = result.to_json(orient = 'columns') # 데이터프레임을 json 형태로 변환
    # print(result_js)
    result_js_list = [result_js] # json 형태의 데이터를 list로 변환
    # print(result_js_list)
    # print(len(result_js_list))
    result_dict[i] = result_js_list # fullcode가 key값이 되고 list 변환한 json이 value값이 됨
    print(len(result_dict)) # 현재 상황을 알기위해 몇개의 종목이 저장되었는지 갯수 확인

In [27]:
url2 = 'http://data.krx.co.kr/comm/bldAttendant/getJsonData.cmd'
headers = {'User-Agent': 'Chrome/78.0.3904.87 Safari/537.36',}

strtDd = '19000101'
endDd = kor_date()

result_dict = {}
for i in fullcode_list[:2]: # 테스트를 위해 2개만 실행 (두 종목만 출력됨)
    make_initialdata()

# 최종 결과물 저장   
with open('/content/drive/MyDrive/k-project/output/initial_data.json', 'w') as f:
    json.dump(result_dict, f)

1
2


In [28]:
print(result_dict)

{'KR7060310000': ['{"TRD_DD":{"0":"2021\\/10\\/05","1":"2021\\/10\\/01","2":"2021\\/09\\/30","3":"2021\\/09\\/29","4":"2021\\/09\\/28","5":"2021\\/09\\/27","6":"2021\\/09\\/24","7":"2021\\/09\\/23","8":"2021\\/09\\/17","9":"2021\\/09\\/16","10":"2021\\/09\\/15","11":"2021\\/09\\/14","12":"2021\\/09\\/13","13":"2021\\/09\\/10","14":"2021\\/09\\/09","15":"2021\\/09\\/08","16":"2021\\/09\\/07","17":"2021\\/09\\/06","18":"2021\\/09\\/03","19":"2021\\/09\\/02","20":"2021\\/09\\/01","21":"2021\\/08\\/31","22":"2021\\/08\\/30","23":"2021\\/08\\/27","24":"2021\\/08\\/26","25":"2021\\/08\\/25","26":"2021\\/08\\/24","27":"2021\\/08\\/23","28":"2021\\/08\\/20","29":"2021\\/08\\/19","30":"2021\\/08\\/18","31":"2021\\/08\\/17","32":"2021\\/08\\/13","33":"2021\\/08\\/12","34":"2021\\/08\\/11","35":"2021\\/08\\/10","36":"2021\\/08\\/09","37":"2021\\/08\\/06","38":"2021\\/08\\/05","39":"2021\\/08\\/04","40":"2021\\/08\\/03","41":"2021\\/08\\/02","42":"2021\\/07\\/30","43":"2021\\/07\\/29","44":"2021\\

In [ ]:
# # Save pickle
# with open(output_path + "3S.pickle","wb") as fw:
#     pickle.dump(result, fw)